The worflow:

for each frame:
    mask cars appearing on frame at instant t (maybe use dilation)
    extract edges (evaluate this two changing the order)
    get Lines filtered by certain range of angles (get horizontal and vertical, but separately) (maybe explore this for lane markings)
Once lines are saved:
        get histogram of orientations
        get histogram of lengths
        remove outliers
map all endpoints of lines to pre defined lengths
use them to train a MLP.
Test
    

In [3]:
import cv2
import numpy as np
import torchvision.transforms as T

from carMasking import *
from utils import *

%matplotlib inline

forExt = DLForegroundExtractor(512)
videoPath = "/home/hector/speed/speed_DS/v1.mp4"

In [15]:
forExt = TraditionalForegroundExtractor()
cap = cv2.VideoCapture("/home/hector/speed/speed_DS/v1.mp4")
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    grayFrame = convertImg2Grayscale(frame)
    foregroundFrame = forExt.maskCarsAtFrame(grayFrame)
    edgeMap = getEdgeMap(foregroundFrame,100,200)
    lines = getLinesFromEdges(edgeMap,(-10,10), 10, 0)
    empty = np.zeros((edgeMap.shape[0],edgeMap.shape[1],3))
    empty[:,:,0] = edgeMap
    linesImg = drawLinesOnImg(empty, lines)
    cv2.imshow('Frame',linesImg)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
  else: 
    break
cap.release()
cv2.destroyAllWindows()

In [5]:
cap = cv2.VideoCapture("/home/hector/speed/speed_DS/v1.mp4")
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    frame = cv2.resize(frame, (512,512))
    grayFrame = convertImg2Grayscale(frame)
    maskedFrame = forExt.segment(frame, grayFrame)
    cv2.imshow('Frame',maskedFrame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
  else: 
    break
cap.release()
cv2.destroyAllWindows()

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [4]:
cv2.imshow("frame", frame)
cv2.waitframe)
    if cv2.waitKey(25) & 0xFF == ord('q'):
          break
            eframe

done
